# Fidelity vs Penalty Energy

Final state fidelity under thermal noise as a function of penalty energy:
- (a) Repetition code [[3,1,3]] with stabilizer penalty
- (b) Bacon-Shor [[4,1,1,2]] with gauge penalty

The simulation uses a piecewise master equation approach with thermal collapse operators.
Parameters are based on IBM dilution fridge conditions.

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, os.path.dirname(os.getcwd()))
from qec_config import (
    QECConfig, BaconShorConfig, PlotConfig, SimulationParams,
    simulate_rap_fidelity, simulate_fidelity_sweep
)

PlotConfig.apply()

# Output directory
OUTPUT_DIR = Path('../figs/thermal_noise')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = Path('../data/thermal_noise')
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Show simulation parameters
SimulationParams.info()

print(f"\nOutput: {OUTPUT_DIR}")
print(f"Data: {DATA_DIR}")

In [ ]:
# Use parameters from SimulationParams (IBM platform)
LAMBDA_2 = SimulationParams.LAMBDA_2
TEMPERATURES = SimulationParams.TEMPERATURES
N_POINTS = SimulationParams.N_POINTS

# Get Ep sweep
Ep_MHz = SimulationParams.get_Ep_sweep_MHz()
Ep_values = SimulationParams.get_Ep_sweep_rad()

# Initialize configurations
rep = QECConfig(n_points=N_POINTS)
bs = BaconShorConfig(n_points=N_POINTS)

print(f"Ep range: {Ep_MHz[0]:.0f} - {Ep_MHz[-1]:.0f} MHz ({len(Ep_MHz)} points)")
print(f"Temperatures: {TEMPERATURES * 1e3} mK")
print(f"lambda_2 = {LAMBDA_2:.0e}")
print(f"Time steps: {N_POINTS}")

In [ ]:
# Simulate repetition code
print("Simulating Repetition Code [[3,1,3]]...")
print("="*60)

rep_fidelities = np.zeros((len(TEMPERATURES), len(Ep_values)))
rep_fidelities_logical = np.zeros((len(TEMPERATURES), len(Ep_values)))

for i, T in enumerate(TEMPERATURES):
    print(f"  T = {T*1e3:.1f} mK ...", end="", flush=True)
    for j, Ep in enumerate(Ep_values):
        result = simulate_rap_fidelity(
            rep, rep.logical_zero, rep.logical_one,
            Ep=Ep, T_K=T, lambda_2=LAMBDA_2,
            n_steps=N_POINTS, verbose=False
        )
        rep_fidelities[i, j] = result['fidelity']
        rep_fidelities_logical[i, j] = result['fidelity_logical']
    print(f" done (avg = {rep_fidelities[i].mean():.4f})")

print("="*60)

In [ ]:
# Simulate Bacon-Shor code
print("Simulating Bacon-Shor Code [[4,1,1,2]]...")
print("="*60)

bs_fidelities = np.zeros((len(TEMPERATURES), len(Ep_values)))
bs_fidelities_logical = np.zeros((len(TEMPERATURES), len(Ep_values)))

for i, T in enumerate(TEMPERATURES):
    print(f"  T = {T*1e3:.1f} mK ...", end="", flush=True)
    for j, Ep in enumerate(Ep_values):
        result = simulate_rap_fidelity(
            bs, bs.logical_zero, bs.logical_one,
            Ep=Ep, T_K=T, lambda_2=LAMBDA_2,
            n_steps=N_POINTS, verbose=False
        )
        bs_fidelities[i, j] = result['fidelity']
        bs_fidelities_logical[i, j] = result['fidelity_logical']
    print(f" done (avg = {bs_fidelities[i].mean():.4f})")

print("="*60)

In [ ]:
# Create publication-quality figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Color map for temperatures
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(TEMPERATURES)))

# Plot repetition code
for i, T in enumerate(TEMPERATURES):
    label = f'{T*1e3:.0f} mK'
    ax1.plot(Ep_MHz, rep_fidelities[i], 'o-', color=colors[i], 
             label=label, markersize=6, linewidth=2)

ax1.set_ylabel(r'Final Fidelity to $|1_L\rangle$', fontsize=16)
ax1.set_title(r'(a) Repetition $[[3,1,3]]$', fontsize=18)
ax1.set_ylim([0, 1.05])
ax1.grid(True, alpha=0.3)
ax1.tick_params(labelsize=14)

# Plot Bacon-Shor code (total fidelity = logical + ancilla)
for i, T in enumerate(TEMPERATURES):
    label = f'{T*1e3:.0f} mK'
    ax2.plot(Ep_MHz, bs_fidelities[i], 's-', color=colors[i], 
             label=label, markersize=6, linewidth=2)

ax2.set_title(r'(b) Bacon-Shor $[[4,1,1,2]]$', fontsize=18)
ax2.grid(True, alpha=0.3)
ax2.tick_params(labelsize=14)

# Shared legend inside right plot
ax2.legend(title='Temperature', fontsize=10, title_fontsize=11, loc='lower right')

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)

# Shared x label
fig.text(0.5, 0.02, r'Penalty Energy $E_p$ [MHz]', ha='center', fontsize=16)

plt.show()

In [ ]:
# Save figure in multiple formats
fig.savefig(OUTPUT_DIR / 'fidelity_vs_Ep_comparison.pdf', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / 'fidelity_vs_Ep_comparison.svg', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / 'fidelity_vs_Ep_comparison.png', dpi=300, bbox_inches='tight')

# Save plot data for reproduction
plot_data = {
    'Ep_MHz': Ep_MHz,
    'temperatures_K': TEMPERATURES,
    'rep_fidelities': rep_fidelities,
    'rep_fidelities_logical': rep_fidelities_logical,
    'bs_fidelities': bs_fidelities,
    'bs_fidelities_logical': bs_fidelities_logical,
    'lambda_2': LAMBDA_2,
    'n_points': N_POINTS
}
np.savez_compressed(OUTPUT_DIR / 'plot_data.npz', **plot_data)

print(f"Saved figures and data to {OUTPUT_DIR}")
print("\nFiles:")
for f in sorted(OUTPUT_DIR.glob('fidelity_vs_Ep*')):
    print(f"  {f.name}")

In [ ]:
# Print summary table
print("\n" + "="*70)
print("SUMMARY: Fidelity at Selected Ep Values")
print("="*70)

# Find indices for specific Ep values
ep_display = [25, 75, 150, 250]
ep_indices = [np.argmin(np.abs(Ep_MHz - e)) for e in ep_display]

print(f"\nRepetition Code [[3,1,3]]:")
print(f"{'Temperature':<12}", end="")
for e in ep_display:
    print(f"  Ep={e:3d} MHz", end="")
print()
print("-"*60)
for i, T in enumerate(TEMPERATURES):
    print(f"{T*1e3:5.1f} mK     ", end="")
    for j in ep_indices:
        print(f"     {rep_fidelities[i,j]:.4f}", end="")
    print()

print(f"\nBacon-Shor Code [[4,1,1,2]] (Total = Logical + Ancilla):")
print(f"{'Temperature':<12}", end="")
for e in ep_display:
    print(f"  Ep={e:3d} MHz", end="")
print()
print("-"*60)
for i, T in enumerate(TEMPERATURES):
    print(f"{T*1e3:5.1f} mK     ", end="")
    for j in ep_indices:
        print(f"     {bs_fidelities[i,j]:.4f}", end="")
    print()

print(f"\nBacon-Shor Code [[4,1,1,2]] (Logical Only):")
print(f"{'Temperature':<12}", end="")
for e in ep_display:
    print(f"  Ep={e:3d} MHz", end="")
print()
print("-"*60)
for i, T in enumerate(TEMPERATURES):
    print(f"{T*1e3:5.1f} mK     ", end="")
    for j in ep_indices:
        print(f"     {bs_fidelities_logical[i,j]:.4f}", end="")
    print()

print("="*70)